# Minesweeper Puzzle

The player is presented with a grid of squares. Some randomly selected squares, unknown to the player, contain mines. Other squares may display numbers indicating the count of mines present in the immediate neighborhood of the squares. Find all mines. 

See https://de.wikipedia.org/wiki/Minesweeper or go to http://minesweeperonline.com/ to play

Imports

In [30]:
from ortools.constraint_solver import pywrapcp
from itertools import product
import numpy as np

Pretty-print square minesweeper board

In [31]:
def pretty_print(mines, labels):
    for i in range(len(labels)):
        for j in range(len(labels)):
            if mines[i][j].Value() == 1:
                print("[💣]\t", end='')
            elif labels[i][j] == "":
                print("[  ]\t", end='')
            else:
                print("[ {}]\t".format(labels[i][j]), end='')
        print("\n")
    print("\n\n")

Some Minesweeper examples, the first one is from the lecture

In [32]:
mine_field1 = [
    ["2", "", "", "", "1"],
    ["2", "", "4", "3", ""],
    ["", "2", "", "1", ""],
    ["", "1", "", "3", ""],
    ["1", "", "", "", ""],
]
mine_field2 = [
    ["1", "", "", "", "2", "", "", "1"],
    ["", "1", "2", "", "3", "", "", "1"],
    ["", "2", "", "1", "", "", "", "0"],
    ["", "2", "1", "", "", "2", "3", "1"],
    ["", "", "", "2", "", "", "", ""],
    ["1", "", "", "", "4", "3", "", ""],
    ["", "1", "", "", "4", "", "3", ""],
    ["1", "", "", "2", "", "2", "", "1"],
]
mine_field3 = [
    ["", "", "", "", "", "1", "", ""],
    ["1", "2", "2", "1", "", "", "1", "1"],
    ["", "1", "", "", "", "", "", ""],
    ["", "", "0", "", "", "1", "", "1"],
    ["", "1", "", "", "", "", "", ""],
    ["", "", "1", "", "2", "3", "", ""],
    ["", "", "", "", "", "", "2", "1"],
    ["", "1", "", "2", "2", "", "2", ""],
]
mine_field4 = [
    ["1", "", "", "2", "", "2", "", "2", "", ""],
    ["", "3", "2", "", "", "", "4", "", "", "1"],
    ["", "", "", "1", "3", "", "", "", "4", ""],
    ["3", "", "1", "", "", "", "3", "", "", ""],
    ["", "2", "1", "", "1", "", "", "3", "", "2"],
    ["", "3", "", "2", "", "", "2", "", "1", ""],
    ["2", "", "", "3", "2", "", "", "2", "", ""],
    ["", "3", "", "", "", "3", "2", "", "", "3"],
    ["", "", "3", "", "3", "3", "", "", "", ""],
    ["", "2", "", "2", "", "", "", "", "2", ""],
]

Pick one of the examples

In [33]:
game = mine_field1

Create constraint solver

In [34]:
solver = pywrapcp.Solver("Minesweeper")

In [35]:
# Create solution_grid
solution_grid = {}
for x in range(len(game)):
    for y in range(len(game[0])):
        solution_grid[(x, y)] = solver.IntVar(0, 1)
        

x_max = len(game)
y_max = len(game[0])

neighbours = [-1, 0, 1]

#iterate over all fields
x_index = 0
y_index = 0
for x in game:
    y_index = 0
    for y in x:
        if game[x_index][y_index] not in "" and int(game[x_index][y_index]) >= 0:
            # assign to 0
            solver.Add(solution_grid[x_index, y_index] == 0)
                            
            solver.Add(
                game[x_index][y_index] == solver.Sum(
                    [solution_grid[x_index + x_offset, y_index + y_offset]
                    for x_offset in neighbours for b in neighbours
                    if x_index + x_offset >= 0 and
                    y_index + y_offset >= 0 and
                    x_index + x_offset < x_max and
                    y_index + y_offset < y_max]
                )
            )
        
        if game[x_index][y_index] in "":
            solver.Add(solution_grid[x_index, y_index] == 0)
        
        y_index = y_index + 1

    x_index = x_index + 1

TypeError: in method 'Solver_AddConstraint', argument 2 of type 'operations_research::Constraint *const'

Configure solver

In [29]:
# Replace this by a list of decision varianles in your model
all_vars = list(np.concatenate(solution_grid))

db = solver.Phase(all_vars, solver.INT_VAR_SIMPLE, solver.INT_VALUE_SIMPLE)

NameError: name 'np' is not defined

Start solver

In [ ]:
solver.NewSearch(db)
while solver.NextSolution():
    print('output solution here')

Cleanup

In [ ]:
solver.EndSearch()

Print solver information

In [ ]:
print("Solutions: {}".format(solver.Solutions()))
print("Runtime:   {}ms".format(solver.WallTime()))
print("Failures:  {}".format(solver.Failures()))
print("Branches:  {} ".format(solver.Branches()))